In [ ]:
dataset_url = "https://www.dropbox.com/s/rouj6qme3dcklk0/initial_sentences_new_unique_rm_unknown.csv?dl=0"
!wget {dataset_url}

--2022-08-05 16:03:34--  https://www.dropbox.com/s/rouj6qme3dcklk0/initial_sentences_new_unique_rm_unknown.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/rouj6qme3dcklk0/initial_sentences_new_unique_rm_unknown.csv [following]
--2022-08-05 16:03:34--  https://www.dropbox.com/s/raw/rouj6qme3dcklk0/initial_sentences_new_unique_rm_unknown.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc363f98d53cc49d1137318e8ccf.dl.dropboxusercontent.com/cd/0/inline/BqcYB10RGoQTW0dBhbqTkHQWPCSLncq_9eHlm-mhCupuaGWtxLrwl2ZFrGq84LFS4FI9TyjNg14GkC_5i8ZRkqxx5ktcuJzELhZ0CG-S06UFXIHzUny7zlNOhsRqY6-igH_dCrFCr-hwyuuxDr1EDJ6BeoqfC28NWYuOZI8qvKQBuA/file# [following]
--2022-08-05 16:03:35--  https://uc363f98d53cc49d1137318e8ccf.dl.dropb

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

!pip install transformers
!pip install datasets

# import needed lib
import numpy as np
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
import json
import torch

from datasets import Dataset, DatasetDict
from datasets import load_metric

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, fbeta_score, precision_score, recall_score,f1_score


import transformers
from transformers import AutoModel, BertTokenizer, BertForPreTraining
from transformers import (AutoTokenizer, AutoConfig,
                          AutoModelForSequenceClassification)
from transformers import Trainer,TrainingArguments

import glob
import random
import datetime
import dask.dataframe as dd



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 2.6 MB/s 
     |████████████████████████████████| 596 kB 50.3 MB/s 
     |████████████████████████████████| 6.6 MB 51.1 MB/s 
     |████████████████████████████████| 101 kB 10.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 3.2 MB/s 
     |████████████████████████████████| 212 kB 63.8 MB/s 
     |████████████████████████████████| 115 kB 51.7 MB/s 
     |████████████████████████████████| 141 kB 55.2 MB/s 
     |████████████████████████████████| 127 kB 53.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully u

In [ ]:
# df = df[:100000]
# df.count

In [ ]:
df = pd.read_csv('/content/initial_sentences_new_unique_rm_unknown.csv?dl=0')
df = df[['text','id']]
df.pop('id')
df_train, df_unsup = train_test_split(df, shuffle=True,test_size=0.2)

# df_train = df_train.sample(n=10000,random_state=1)
ds= DatasetDict({
    "train": Dataset.from_pandas(df_train.reset_index(drop=True)),
    "unsup": Dataset.from_pandas(df_unsup.reset_index(drop=True))})

model_ckpt = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True,
                     max_length=20, return_special_tokens_mask=True)

ds_mlm = ds.map(tokenize, batched=True)

ds_mlm = ds_mlm.remove_columns(["text"])


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


  0%|          | 0/759 [00:00<?, ?ba/s]

  0%|          | 0/190 [00:00<?, ?ba/s]

In [ ]:
ds_mlm

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 758455
    })
    unsup: Dataset({
        features: ['input_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 189614
    })
})

In [ ]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your termin

In [ ]:
from transformers import AutoModelForMaskedLM
from transformers import DataCollatorForLanguageModeling, set_seed

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,
                                                mlm_probability=0.15)

training_args = TrainingArguments(
    output_dir = f"{model_ckpt}-EnglishLawAI_roberta_base_version_bg", 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=32,
    learning_rate=5e-5,
    logging_strategy="epoch", evaluation_strategy="epoch", save_strategy="no",
    num_train_epochs=16, push_to_hub=True, log_level="error", report_to="none")

trainer = Trainer(
        model=AutoModelForMaskedLM.from_pretrained("roberta-base"),
        tokenizer=tokenizer, args=training_args, data_collator=data_collator,
        train_dataset=ds_mlm["train"], eval_dataset=ds_mlm["unsup"])

trainer.train()

Cloning https://huggingface.co/Makabaka/roberta-base-EnglishLawAI_roberta_base_version3 into local empty directory.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss
1,2.642800,2.435148
2,2.514700,2.381908


In [ ]:
# learning rate
# training sample size


In [ ]:
trainer.push_to_hub("Training complete!")

df_log = pd.DataFrame(trainer.state.log_history)

(df_log.dropna(subset=["eval_loss"]).reset_index()["eval_loss"]
 .plot(label="Validation"))
df_log.dropna(subset=["loss"]).reset_index()["loss"].plot(label="Train")

plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(loc="upper right")
plt.show()